In [7]:
import os
os.chdir('../')

import transtab

# set random seed
transtab.random_seed(42)

In [8]:
# load a dataset and start vanilla supervised training
allset, trainset, valset, testset, cat_cols, num_cols, bin_cols \
    = transtab.load_data('credit-g')

########################################


/root/transtab/transtab/transtab/dataset.py:167: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = openml.datasets.get_dataset(dataname)


openml data index: 31
load data from credit-g


NameError: free variable 'bin_cols' referenced before assignment in enclosing scope

In [ ]:
# make a fast pre-train of TransTab contrastive learning model
# build contrastive learner, set supervised=True for supervised VPCL
model, collate_fn = transtab.build_contrastive_learner(
    cat_cols, num_cols, bin_cols, 
    supervised=True, # if take supervised CL
    num_partition=4, # num of column partitions for pos/neg sampling
    overlap_ratio=0.5, # specify the overlap ratio of column partitions during the CL
)

# start contrastive pretraining training
training_arguments = {
    'num_epoch':50,
    'batch_size':64,
    'lr':1e-4,
    'eval_metric':'val_loss',
    'eval_less_is_better':True,
    'output_dir':'./checkpoint'
    }

transtab.train(model, trainset, valset, collate_fn=collate_fn, **training_arguments)

In [ ]:
# There are two ways to build the encoder
# First, take the whole pretrained model and output the cls token embedding at the last layer's outputs
enc = transtab.build_encoder(
    binary_columns=bin_cols,
    checkpoint = './checkpoint'
)

In [ ]:
# Then take the encoder to get the input embedding
df = trainset[0]
output = enc(df)
print(output.shape)
output[:2]

In [ ]:
df.head()

In [ ]:
# Second, if we only want to the embeded token level embeddings (embeddings before going to transformers)
enc = transtab.build_encoder(
    binary_columns=bin_cols,
    checkpoint = './checkpoint',
    num_layer = 0,
)

In [ ]:
output = enc(df)
print(output['embedding'].shape)
output['embedding'][:2]